In [ ]:
from pathlib import Path

import pandas as pd

from blooms_ml.utils import (
    add_previous,
    get_stats,
)

Open the prepared from ROHO800 model output data and label it

In [ ]:
datadir = f"{Path.home()}/data_ROHO"
df = pd.read_parquet(f"{datadir}/roho800_weekly_average.parquet")
(p1_c_mean, n1_p_mean, n3_n_mean, n5_s_mean,
 p1_c_std, n1_p_std, n3_n_std, n5_s_std) = get_stats(f"{datadir}/cnps_mean_std.csv")

"Normalization"

In [ ]:
df['P1_c'] = ((df['P1_c'] - float(p1_c_mean)) / float(p1_c_std)).round(2).astype('float32')
df['N1_p'] = ((df['N1_p'] - float(n1_p_mean)) / float(n1_p_std)).round(2).astype('float32')
df['N3_n'] = ((df['N3_n'] - float(n3_n_mean)) / float(n3_n_std)).round(2).astype('float32')
df['N5_s'] = ((df['N5_s'] - float(n5_s_mean)) / float(n5_s_std)).round(2).astype('float32')
df['rho'] = ((df['rho'] - df['rho'].mean()) / df['rho'].std()).round(2).astype('float32')

In [ ]:
df = df.groupby(['station', 's_rho']).apply(add_previous, include_groups=False)
df = df.reset_index().drop(columns='level_2')

In [ ]:
df.isna().any().any()

In [ ]:
df

In [ ]:
df.to_parquet(f"{Path.home()}/data_ROHO/roho800_weekly_average_stacked.parquet", index=False, engine='pyarrow')